In [36]:
from inspect import Signature, Parameter

def r_dataclass(cls):
    # grab the annotation dict
    ann = cls.__annotations__
    # make a signature out of the keys of annotation dict
    signature = Signature([Parameter(field, Parameter.POSITIONAL_OR_KEYWORD) for field in ann])
    # define __init__ that is injected 
    print(" phase I")
    dota = " yay i figured it out"
    def __init__(self, *args, **kwargs):
		#bind the signature to arguments of constructor
        print(" Phase II")
        print(dota)
        bounded = signature.bind(*args, **kwargs)
        for key, val in bounded.arguments.items():
			# set the key and value in the instance of give class 
            setattr(self, key, val)
    setattr(cls, '__init__', __init__)
    print(" Phase III")
    return cls

# the way you use it
@r_dataclass
class Vector:
  x: float
  y: float

@r_dataclass
class Person:
  name: str
  age: int


if __name__ == '__main__':
    print( " What phase is running ?")
    v = Vector(3.0, 4.3)
    print(v.x, v.y)


 phase I
 Phase III
 phase I
 Phase III
 What phase is running ?
 Phase II
 yay i figured it out
3.0 4.3


In [29]:
class retroFit:
    x: float
    y = 't'

In [24]:
ret = retroFit
ret.x = 8.77
ret.x

8.77

In [30]:
ann = retroFit.__annotations__
ann

{'x': float}

In [9]:
signature = Signature([Parameter(field, Parameter.POSITIONAL_OR_KEYWORD) for field in ann])
signature

<Signature (x, y)>

In [16]:
def outer_decorator(*outer_args,**outer_kwargs):                            
    def decorator(fn):                                            
        def decorated(*args,**kwargs):                            
            print(*outer_args,**outer_kwargs)                      
            return fn(*args,**kwargs)                         
        return decorated                                          
    return decorator       
    
@outer_decorator(1,2,3)
def foo(a,b,c):
    print (a)
    print (b)
    print (c)


foo(3,8,0)

1 2 3
3
8
0


In [20]:
def time_this(original_function):      
    print ("decorating")                      
    def new_function(*args,**kwargs):
        print ("starting timer")       
        import datetime                 
        before = datetime.datetime.now()                     
        x = original_function(*args,**kwargs)                
        after = datetime.datetime.now()                      
        print ("Elapsed Time = {0}".format(after-before))      
        return x                                             
    return new_function  

def time_all_class_methods(Cls):
    class NewCls(object):
        def __init__(self,*args,**kwargs):
            self.oInstance = Cls(*args,**kwargs)
        def __getattribute__(self,s):
            """
            this is called whenever any attribute of a NewCls object is accessed. This function first tries to 
            get the attribute off NewCls. If it fails then it tries to fetch the attribute from self.oInstance (an
            instance of the decorated class). If it manages to fetch the attribute from self.oInstance, and 
            the attribute is an instance method then `time_this` is applied.
            """
            try:    
                x = super(NewCls,self).__getattribute__(s)
            except AttributeError:      
                pass
            else:
                return x
            x = self.oInstance.__getattribute__(s)
            if type(x) == type(self.__init__): # it is an instance method
                return time_this(x)                 # this is equivalent of just decorating the method with time_this
            else:
                return x
    return NewCls

#now lets make a dummy class to test it out on:

@time_all_class_methods
class Foo(object):
    def a(self):
        print ("entering a")
        import time
        time.sleep(3)
        print ("exiting a")

oF = Foo()
oF.a()

decorating
starting timer
entering a
exiting a
Elapsed Time = 0:00:03.000282


In [1]:
x = input("Do you need the answer? (y/n): ")
if x.lower() == "y":
    required = True
else:
    required = False
    
def the_answer(self, *args):              
        return 42
    
class EssentialAnswers(type):
    
    def __init__(cls, clsname, superclasses, attributedict):
        if required:
            print(clsname)
            cls.the_answer = the_answer
                           
    
class Philosopher1(metaclass=EssentialAnswers): 
    pass
class Philosopher2(metaclass=EssentialAnswers): 
    pass
class Philosopher3(metaclass=EssentialAnswers): 
    pass
    
    
plato = Philosopher1()
print(plato.the_answer())
kant = Philosopher2()
# let's see what Kant has to say :-)
print(kant.the_answer())

Do you need the answer? (y/n): y
Philosopher1
Philosopher2
Philosopher3
42
42
